In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import datetime
import requests
import gmaps
import os
import pycountry_convert as pc

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [45]:
# Make a reference to the starter csv file path and create df
pprep_df = pd.read_csv('Resources/High_Value_Dataset.csv') 
pprep_df

# # Import the offense category csv file as a DataFrame, utf-8 coding
# # csv_path = "Resources/proj_start.csv"
# offense_df = pd.read_csv('Resources/offenseCodes_csv_ut8.csv', encoding="utf-8")
# offense_df.head()

# # Combine the data into a single dataset
# # mscomb = mouse_metadata.merge(study_results, on = "Mouse ID", how = "outer")

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,068770,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS
1,524524,758486,"HARPER,DAVID JOHN JR",Estelle,M,W,91,01/01/9999,01/01/9999,08/03/2031,95-765-C,McLennan,36010001,INDECENCY WITH A CHILD,08/13/1996,09/15/1995,Life,None,08/03/2031,NOT IN REVIEW PROCESS
2,703267,841623,"RODRIGUEZ,THEODORE",Beto,M,W,82,08/03/2043,08/03/2043,02/01/2021,97-396-C,McLennan,11990003,AGG SEX ASLT,08/05/1998,06/01/1996,45.0,None,02/01/2021,IN PAROLE REVIEW PROCESS
3,708520,256174,"ALONZO,BENITO",Hospital Galveston,M,H,85,01/01/9999,01/01/9999,11/25/1985,49995,Travis,49030000,INTRODUCE DEADLY WPN PENAL INSTI,02/27/1976,06/01/1974,Life,Denied on 12/20/2019,12/2020,IN PAROLE REVIEW PROCESS
4,766785,365547,"BROWN,NARRIES EARL",Jester IV,M,W,84,01/01/9999,01/01/9999,08/24/2003,F83-89728-HI,Dallas,11220000,AGG SEX ABUSE CHILD/U14,08/24/1983,03/15/1983,Life,Denied on 12/05/2016,Unavailable at this time.,NOT IN REVIEW PROCESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120702,50824296,2192344,"PINEDA-NAVA,CRISTIAN",Clemens,M,H,22,01/20/2056,01/20/2056,01/20/2036,241-0316-16,Smith,11990004,AGG SEXUAL ASLT CHILD,02/22/2018,10/15/2015,40.0,None,01/20/2036,NOT IN REVIEW PROCESS
120703,50824299,2180043,"AARON,LYNDSEY NICOLE",Crain,F,W,22,09/18/2021,09/18/2021,NaN,D-1-DC-17-300046,Travis,13990063,ASLT PUB SERV,02/09/2018,01/03/2017,4.0,Denied on 03/09/2020,03/2021,NOT IN REVIEW PROCESS
120704,50824431,2173750,"WRIGHT,LARRY LEE",Robertson,M,W,73,01/15/2043,01/15/2043,07/16/2030,CR02467,Red River,11990004,AGG SEX ASLT CHILD,01/17/2018,01/15/2016,25.0,None,07/16/2030,NOT IN REVIEW PROCESS
120705,50824569,2272406,"COX,CARRIE BETH",San Saba,F,W,25,04/18/2024,04/18/2024,11/15/2019,CR-1090-19-D,Hidalgo,13150004,AGG ASLT SBI,07/10/2019,01/19/2019,5.0,Denied on 11/25/2019,11/2020,IN PAROLE REVIEW PROCESS


In [3]:
# Import the offense category csv file as a DataFrame, utf-8 coding
# csv_path = "Resources/proj_start.csv"
offense_df = pd.read_csv('Resources/offenseCodes_csv_ut8.csv', encoding="utf-8")
offense_df

# Combine the data into a single dataset
# mscomb = mouse_metadata.merge(study_results, on = "Mouse ID", how = "outer")

,Code,Offense_Category,Offense,Citation,Statute,L/D
0,3990001,100,MISREPRESENT CHILD AS FAMILY MEMBER ENTRY PORT,37.082(c),PC,MB
1,64110001,100,SMUGGLING OF PERSONS,20.05(b),PC,F3
2,64110002,100,SMUGGLING OF PERSONS: LIKELIHOOD SBI OR DEATH,20.05(b)(1)(A),PC,F2
3,64110003,100,SMUGGLING OF PERSONS: UNDER 18 YOA,20.05(b)(1)(B),PC,F2
4,64110004,100,SMUGGLING OF PERSONS:BECAME SX ASSLT VICT IAT,20.05(b)(2)(A),PC,F1
...,...,...,...,...,...,...
2354,73990717,6200,OPERATION OF STASH HOUSE,20.07(b),PC,MA
2355,73991073,6200,DIRECT ACTIVITIES OF STREET GANG,71.023,PC,F1
2356,73991081,6200,COERCE/SOLICIT/INDUCE GANG MEMBERSHIP THREAT,71.022(a-1)(1),PC,F3
2357,73991082,6200,COERCE/SOLICIT/INDUCE GANG MEMBERSHIP CAUSE BI,71.022(a-1)(2),PC,F3


In [4]:
# Create project specific df for merging
offproject_df = offense_df.loc[:,["Code", "Offense_Category"]]
offproject_df

,Code,Offense_Category
0,3990001,100
1,64110001,100
2,64110002,100
3,64110003,100
4,64110004,100
...,...,...
2354,73990717,6200
2355,73991073,6200
2356,73991081,6200
2357,73991082,6200


In [5]:
newoffocde_df = offproject_df.rename(columns={"Code": "Offense Code"})

In [6]:
newoffocde_df

,Offense Code,Offense_Category
0,3990001,100
1,64110001,100
2,64110002,100
3,64110003,100
4,64110004,100
...,...,...
2354,73990717,6200
2355,73991073,6200
2356,73991081,6200
2357,73991082,6200


In [7]:
pprep_df.isna().sum()

SID Number                     0
TDCJ Number                    0
Name                           0
Current Facility               0
Gender                         0
Race                           0
Age                            0
Projected Release              0
Maximum Sentence Date         28
Parole Eligibility Date     9017
Case Number                   99
County                         0
Offense Code                   0
TDCJ Offense                   0
Sentence Date               1233
Offense Date                   0
Sentence (Years)               1
Last Parole Decision        7815
Next Parole Review Date    22397
Parole Review Status        7815
dtype: int64

In [8]:
pprep_df.dtypes

SID Number                  int64
TDCJ Number                 int64
Name                       object
Current Facility           object
Gender                     object
Race                       object
Age                         int64
Projected Release          object
Maximum Sentence Date      object
Parole Eligibility Date    object
Case Number                object
County                     object
Offense Code                int64
TDCJ Offense               object
Sentence Date              object
Offense Date               object
Sentence (Years)           object
Last Parole Decision       object
Next Parole Review Date    object
Parole Review Status       object
dtype: object

In [9]:
dupSID = pprep_df[pprep_df.duplicated(["SID Number"])]
dupSID.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status


In [10]:
dupTDJCnum = pprep_df[pprep_df.duplicated(["TDCJ Number"])]
dupTDJCnum.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status


In [11]:
dupName = pprep_df[pprep_df.duplicated(["Name"])]
dupName.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
2376,2061956,2009153,"HERNANDEZ,JOHN",Clements,M,H,65,10/01/2046,07/18/2094,09/30/2020,F-2014-1924-A,Denton,54040011,DWI,04/22/2015,07/18/2014,80.0,Denied on 08/11/2020,08/2021,NOT IN REVIEW PROCESS
2469,2082179,596988,"VILLARREAL,RAUL",Michael,M,H,63,01/09/2021,01/09/2021,04/10/1998,91-CR-1059,Bexar,12120000,ROBBERY/THREATS,09/16/1991,01/09/1991,30.0,Denied on 09/08/2020,NaN,NOT IN REVIEW PROCESS
2601,2114848,1858995,"SMITH,EUGENE",Jordan,M,B,62,01/21/2043,01/21/2043,03/05/2016,137465901010,Harris,12990001,ROBBERY-THREATS,05/09/2013,01/21/2013,30.0,Denied on 03/12/2019,03/2021,NOT IN REVIEW PROCESS
3657,2343592,827821,"ESPARZA,EDWARD",Polunsky,M,H,64,07/29/2062,07/29/2062,07/29/2027,97CR0690,Bexar,9990001,MURDER,02/17/1998,08/03/1996,65.0,None,07/29/2027,NOT IN REVIEW PROCESS
4187,2437996,814962,"WILLIAMS,LARRY",Clements,M,B,64,06/24/2047,06/24/2047,06/24/2022,0660903D,Tarrant,11990001,SEX ASLT,01/13/1998,05/16/1997,50.0,None,06/24/2022,NOT IN REVIEW PROCESS


In [12]:
dupName = pprep_df[pprep_df.duplicated(["Name", "SID Number"])]
dupName.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status


In [13]:
dupName = pprep_df[pprep_df.duplicated(["Name", "TDCJ Number"])]
dupName.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status


In [14]:
dupName = pprep_df[pprep_df.duplicated(["Name", "Current Facility"])]
dupName.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
5887,2687259,841884,"GARZA,RUBEN",Ramsey,M,H,59,05/09/2048,05/09/2048,05/10/2023,12763,Wharton,11990004,AGG SEX ASLT CHILD,09/28/1998,05/10/1998,50.0,None,05/10/2023,NOT IN REVIEW PROCESS
7024,2847202,2274718,"RODRIGUEZ,BENITO",Smith,M,H,57,06/01/2021,06/10/2024,04/01/2018,01-CR-0923-E,Cameron,13990006,INJ TO CHILD,10/19/2001,05/14/2001,7.0,Denied on 06/22/2020,06/2021,NOT IN REVIEW PROCESS
7257,2879091,920378,"BROWN,KENNETH RAY",Allred,M,B,59,01/01/9999,01/01/9999,05/24/2029,F-9902178-UN,Dallas,13150005,ATT AGG ASLT,10/22/1999,05/25/1999,Life,None,05/24/2029,NOT IN REVIEW PROCESS
8999,3094401,1324657,"JONES,KEVIN MICHAEL",Allred,M,W,57,11/10/2068,11/10/2068,11/10/2033,401-80101-04,Collin,9990019,MURDER,08/24/2005,11/11/2003,65.0,None,11/10/2033,NOT IN REVIEW PROCESS
11495,3413890,1515228,"JONES,DARRELL",Polunsky,M,B,53,06/07/2022,06/07/2022,12/07/2014,F-0753291-P,Dallas,35990003,POSS W/I DEL C/S COCAINE 4G,07/01/2008,06/08/2007,15.0,Denied on 05/14/2020,05/2021,NOT IN REVIEW PROCESS


In [15]:
#Visualize dup data
rgpprep_df = pprep_df.loc[pprep_df["Name"] == "GARZA,RUBEN"]
rgpprep_df

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
548,1483698,1523343,"GARZA,RUBEN",Hughes,M,H,86,04/11/2038,04/11/2038,04/12/2023,07-12-00157-CRK,Karnes,11990004,AGG SEX ASLT CHILD,08/13/2008,04/11/2007,30.0,None,04/12/2023,NOT IN REVIEW PROCESS
5208,2591092,689305,"GARZA,RUBEN",Ramsey,M,H,61,01/01/9999,01/01/9999,02/28/2008,93-M-092,Matagorda,11180000,AGG SEX ASLT,11/16/1994,02/22/1993,Life,Denied on 04/17/2020,03/2023,NOT IN REVIEW PROCESS
5887,2687259,841884,"GARZA,RUBEN",Ramsey,M,H,59,05/09/2048,05/09/2048,05/10/2023,12763,Wharton,11990004,AGG SEX ASLT CHILD,09/28/1998,05/10/1998,50.0,None,05/10/2023,NOT IN REVIEW PROCESS
19831,4207273,1826136,"GARZA,RUBEN",Darrington,M,H,55,10/01/2021,09/04/2030,08/02/2014,12-CR-3405-H,Nueces,13990075,ASLT(FAM VIOL PRIOR)(REPEAT),12/06/2012,04/04/2012,18.0,Approved on 09/23/2020,NaN,NOT IN REVIEW PROCESS
25483,4633451,2071920,"GARZA,RUBEN",Telford,M,H,46,02/05/2021,10/21/2024,01/21/2018,16-CRD-82,Duval,35990003,POSS CS W/INT/DEL,08/07/2017,07/11/2015,8.0,Denied on 04/15/2020,02/05/2021,IN PAROLE REVIEW PROCESS
38801,5560587,2291837,"GARZA,RUBEN",Jordan,M,H,37,09/05/2024,09/05/2024,03/09/2022,2019CR10796,Bexar,12990002,AGG ROBBERY,10/24/2019,07/04/2019,5.0,None,03/09/2022,NOT IN REVIEW PROCESS
40640,5632960,2102105,"GARZA,RUBEN",Roach,M,W,36,08/06/2022,08/06/2022,08/07/2018,134209701010,Harris,36010001,INDEC W/CHILD-TG,10/21/2016,10/14/2011,8.0,Denied on 09/27/2018,09/2021,NOT IN REVIEW PROCESS


In [16]:
pprep_df.nunique()

SID Number                 120707
TDCJ Number                120707
Name                       116346
Current Facility              106
Gender                          2
Race                            7
Age                            75
Projected Release           17895
Maximum Sentence Date       18845
Parole Eligibility Date     15941
Case Number                119459
County                        253
Offense Code                  746
TDCJ Offense                17680
Sentence Date                8852
Offense Date                12417
Sentence (Years)              278
Last Parole Decision         2693
Next Parole Review Date      9055
Parole Review Status            2
dtype: int64

In [17]:
offcodeoff_df = pprep_df[[ "Offense Code", "TDCJ Offense"]]
soffcodeoff_df = offcodeoff_df.sort_values(by=["Offense Code"])
soffcodeoff_df.head(20)

,Offense Code,TDCJ Offense
33752,9000001,MURDER
24056,9000001,MURDER
34826,9000001,MURDER
30561,9000001,MURDER
51593,9000001,MURDER
2195,9000001,MURDER
13130,9000001,SOLICIT COMM CAP MURDER
32623,9000002,MURDER
34677,9000002,MURDER
36239,9000002,MURDER


In [18]:
offcode_df = pprep_df[[ "Offense Code"]]
soffcode_df = offcode_df.sort_values(by=["Offense Code"])
soffcode_df.head()

,Offense Code
33752,9000001
24056,9000001
34826,9000001
30561,9000001
51593,9000001


In [33]:
offcodeseries = pd.Series(soffcode_df["Offense Code"].values)
offarray = offcodeseries.unique()
offarray

array([ 9000001,  9000002,  9000003,  9000005,  9000016,  9000017,
        9070000,  9120000,  9130000,  9140000,  9150000,  9160000,
        9170000,  9180000,  9190000,  9990000,  9990001,  9990002,
        9990003,  9990004,  9990005,  9990007,  9990008,  9990009,
        9990010,  9990011,  9990012,  9990014,  9990015,  9990016,
        9990017,  9990018,  9990019,  9990020,  9990021,  9990022,
        9990023,  9990024,  9990025,  9990026,  9990027,  9990028,
        9990029,  9990030, 10100000, 10110000, 10990001, 10990002,
       10990003, 10990004, 10990005, 10990007, 10990009, 10990011,
       10990013, 10990014, 10990015, 10990016, 10990017, 10990020,
       11000001, 11000002, 11000006, 11020000, 11030000, 11180000,
       11190000, 11200000, 11220000, 11990000, 11990001, 11990002,
       11990003, 11990004, 11990005, 11990006, 11990008, 11990009,
       11990010, 11990011, 11990012, 11990015, 11990016, 11990017,
       12000001, 12120000, 12130000, 12140000, 12150000, 12990

In [20]:
# Checking the for NAN, getting number of unique mice, making sure rows match, etc.
#mscomb.isna().sum()

# ---> .isna().sum()
# Mouse ID              0
# Drug Regimen          0
# Sex                   0
# Age_months            0
# Weight (g)            0
# Timepoint             0
# Tumor Volume (mm3)    0
# Metastatic Sites      0
# dtype: int64

# ---> .nunique()  ----> 249 unique mice
#.unique will give an array of the unique mice

# ---> .count()  --> number of each rows
# Mouse ID              1893
# Drug Regimen          1893
# Sex                   1893
# Age_months            1893
# Weight (g)            1893
# Timepoint             1893
# Tumor Volume (mm3)    1893
# Metastatic Sites      1893
# dtype: int64

# --> .describe()
#Age months, weight, timepoint, tumor volume, metastatic sites - numbers

In [44]:
combined3 = pd.merge_ordered(pprep_df, newoffocde_df, fill_method='ffill', left_by='Offense Code')
combined3

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status,Offense_Category
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,...,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200.0
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,05/02/2021,06/18/2033,03/08/2016,...,Dallas,35990003,UNLAW POSS WIT C/S-METHAM,04/06/2006,09/03/2004,25.0,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200.0
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,01/01/9999,01/01/9999,08/22/2020,...,Williamson,35990003,DEL C/S,09/19/1995,09/28/1994,Life,Denied on 07/16/2020,07/2021,NOT IN REVIEW PROCESS,2200.0
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,04/21/2028,03/11/2043,10/20/2020,...,Grayson,35990003,POSS CONT SUB PG1 4-200 WITD,03/12/2018,03/31/2017,25.0,Denied on 08/13/2020,08/2021,NOT IN REVIEW PROCESS,2200.0
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,08/16/2025,06/27/2040,02/14/2018,...,Johnson,35990003,DEL CONT SUB 4G-200G,11/10/2015,05/05/2015,25.0,Denied on 02/06/2020,02/2021,NOT IN REVIEW PROCESS,2200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120702,50538037,2250200,"DAVIS,ZAKAYLA MONAENYCOLE",Hilltop,F,B,20,06/22/2023,06/22/2023,06/21/2020,...,Dallas,64110014,TRAFFICK CHILD WI FORCED LABOR,02/18/2019,06/22/2018,5.0,Denied on 05/28/2020,05/2022,NOT IN REVIEW PROCESS,100.0
120703,50580998,2087444,"RICKETTS,DUSTIN LEE",Connally,M,W,24,12/09/2022,10/05/2023,04/02/2017,...,Cherokee,22060005,UNL USE OF CRIMINAL INTS,02/04/2016,10/07/2014,8.0,Denied on 02/07/2017,Unavailable at this time.,NOT IN REVIEW PROCESS,1400.0
120704,50709066,2144543,"WERKMEISTER,DEBORAH JANE",Hilltop,F,W,55,11/01/2020,06/22/2027,07/23/2018,...,Ellis,23990093,THEFT BY PUBLIC SERVANT,06/23/2017,05/01/2009,10.0,Approved on 06/12/2020,NaN,NOT IN REVIEW PROCESS,NaN
120705,50776498,2298374,"NINO-RUIZ,JOSE EDUARDO",Lychner,M,H,29,12/05/2021,12/05/2021,NaN,...,Starr,36060001,BESTIALITY,12/09/2019,03/15/2019,2.0,NaN,NaN,NaN,400.0


In [48]:
combined3.isna().sum()

SID Number                     0
TDCJ Number                    0
Name                           0
Current Facility               0
Gender                         0
Race                           0
Age                            0
Projected Release              0
Maximum Sentence Date         28
Parole Eligibility Date     9017
Case Number                   99
County                         0
Offense Code                   0
TDCJ Offense                   0
Sentence Date               1233
Offense Date                   0
Sentence (Years)               1
Last Parole Decision        7815
Next Parole Review Date    22397
Parole Review Status        7815
Offense_Category           27292
dtype: int64

In [51]:
combined4 = combined3[combined3['Offense_Category'].isnull()]
combined4.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status,Offense_Category
4808,524524,758486,"HARPER,DAVID JOHN JR",Estelle,M,W,91,01/01/9999,01/01/9999,08/03/2031,...,McLennan,36010001,INDECENCY WITH A CHILD,08/13/1996,09/15/1995,Life,None,08/03/2031,NOT IN REVIEW PROCESS,NaN
4809,789204,1346013,"RAMOS,JESUS CISNEROS",Connally,M,H,89,09/30/2020,08/28/2027,08/27/2016,...,Bexar,36010001,INDECENCY W-CHILD,12/05/2005,05/05/2004,22.0,Approved on 03/30/2020,NaN,NOT IN REVIEW PROCESS,NaN
4810,997481,2021112,"YBARRA,ERENIO S",C.T. Terrell,M,H,79,05/31/2029,05/31/2029,02/22/2016,...,Harris,36010001,ATT INDECENCY CHILD TOUCHING,08/03/2015,06/01/2014,15.0,Denied on 06/26/2020,06/2021,NOT IN REVIEW PROCESS,NaN
4811,1095776,2202166,"HIDROGO,RUDULFO",Stringfellow,M,H,76,11/13/2024,11/13/2024,11/14/2020,...,Travis,36010001,IND W/CHILD SEX CONT,05/30/2018,07/22/2016,8.0,None,11/14/2020,IN PAROLE REVIEW PROCESS,NaN
4812,1103326,2269694,"INGRAM,CHARLES CLYDE",Powledge,M,W,81,02/01/2025,02/01/2025,08/03/2021,...,Henderson,36010001,INDEC W/CHILD SEX CONTACT,03/05/2019,06/01/2006,7.0,None,08/03/2021,NOT IN REVIEW PROCESS,NaN


In [52]:
combined4.nunique()

SID Number                 27292
TDCJ Number                27292
Name                       26959
Current Facility             104
Gender                         2
Race                           6
Age                           73
Projected Release          11158
Maximum Sentence Date      12136
Parole Eligibility Date    11920
Case Number                27176
County                       241
Offense Code                 379
TDCJ Offense                4450
Sentence Date               8231
Offense Date               10739
Sentence (Years)             185
Last Parole Decision        1993
Next Parole Review Date     6125
Parole Review Status           2
Offense_Category               0
dtype: int64

In [70]:
c4offcode_df = combined4[[ "Offense Code","TDCJ Offense"]]
c4soffcode_df = c4offcode_df.sort_values(by=["Offense Code"])
c4soffcode_df.to_csv('output/c4offcode.csv', index=False)
c4soffcode_df.head(50)

,Offense Code,TDCJ Offense
112918,9000001,MURDER
112919,9000001,SOLICIT COMM CAP MURDER
112920,9000001,MURDER
112921,9000001,MURDER
112922,9000001,MURDER
112923,9000001,MURDER
112924,9000001,MURDER
118794,9000002,MURDER
118791,9000002,MURDER
118792,9000002,MURDER


In [68]:
c4offcodeseries = pd.Series(c4soffcode_df["Offense Code"].values)
# c4offcodeseries.head(50)
c4offarray = c4offcodeseries.unique()
c4offarray

array([ 9000001,  9000002,  9000003,  9000005,  9000016,  9000017,
        9070000,  9120000,  9130000,  9140000,  9150000,  9160000,
        9170000,  9180000,  9190000,  9990000,  9990001,  9990002,
        9990003,  9990004,  9990005,  9990007,  9990008,  9990009,
        9990010,  9990011,  9990012,  9990014,  9990015,  9990016,
        9990019,  9990027,  9990028, 10100000, 10110000, 10990003,
       10990004, 10990005, 11000001, 11000002, 11000006, 11020000,
       11030000, 11180000, 11190000, 11200000, 11220000, 11990000,
       11990005, 11990006, 11990009, 11990011, 11990012, 11990016,
       11990017, 12000001, 12120000, 12130000, 12140000, 12150000,
       12990000, 13000002, 13000004, 13000005, 13000030, 13000031,
       13000036, 13040000, 13070000, 13080000, 13140000, 13150000,
       13150001, 13150002, 13160009, 13160010, 13160011, 13170000,
       13180000, 13190000, 13200000, 13210000, 13990000, 13990002,
       13990003, 13990006, 13990008, 13990009, 13990011, 13990

In [72]:
combined3["Offense_Category"].unique()

array([2200.,   nan,  400.,  500., 4200.,  600.,  200., 1500.,  900.,
       1800., 3500., 4600., 4500., 3700., 3400., 3100., 3200., 1600.,
       1900.,  300., 3600., 1200., 2300., 2700., 2900., 2400., 4900.,
       3300., 4100., 1100., 3900., 2500., 4300.,  100., 2100., 6200.,
       1400., 5200., 4700., 5500., 6100., 5400.,  700.])

In [ ]:
# Store 'Lat' and 'Lng' into  locations 
locations = vac_df[["Latitude", "Longitude"]].astype(float)

# Weight
humheat = vac_df["Humidity_Percent"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humheat, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig